In [1]:
%run basics
%matplotlib
import pysolar
import pytz

Using matplotlib backend: Qt4Agg


In [ ]:
def solar_altitude_IMcH():
    pass


In [2]:
ecmwf_name = "../ECMWF/2014.nc"
ecmwf_file = netCDF4.Dataset(ecmwf_name)

In [3]:
latitude = ecmwf_file.variables["latitude"][:]
longitude = ecmwf_file.variables["longitude"][:]
# [0,0] is top left
#print latitude[0],longitude[0]
#print latitude[-1],longitude[-1]
lat_resolution = abs(latitude[-1]-latitude[0])/(len(latitude)-1)
lon_resolution = abs(longitude[-1]-longitude[0])/(len(longitude)-1)
#print lat_resolution,lon_resolution

In [4]:
# get the time and convert to Python datetime object
ecmwf_time = ecmwf_file.variables["time"][:]
time_units = getattr(ecmwf_file.variables["time"],"units")
dt_ecmwf =  netCDF4.num2date(ecmwf_time,time_units)
hour_utc = numpy.array([dt.hour for dt in dt_ecmwf])
#print dt_ecmwf[0],dt_ecmwf[-1]

In [5]:
# variables are dimensioned [time,latitude,longitude]
Fsd_3d = ecmwf_file.variables["ssrd"][:,:,:]

In [6]:
site_name = "Whroo"
site_timezone = "Australia/Melbourne"
site_latitude = -36.67305
site_longitude = 145.02621
site_lat_index = int(((latitude[0]-site_latitude)/lat_resolution)+0.5)
site_lon_index = int(((site_longitude-longitude[0])/lon_resolution)+0.5)
print site_lat_index,site_lon_index,latitude[site_lat_index],longitude[site_lon_index]

36 47 -36.75 144.75


In [7]:
Fsd_accum = Fsd_3d[:,site_lat_index,site_lon_index]

In [18]:
site_tz = pytz.timezone(site_timezone)
# make utc_dt timezone aware
dt_ecmwf_utc = [x.replace(tzinfo=pytz.utc) for x in dt_ecmwf]
# get local time from UTC
dt_ecmwf_loc = [x.astimezone(site_tz) for x in dt_ecmwf_utc]
# NOTE: will have to disable daylight saving at some stage, towers stay on Standard Time
# PRI hopes that the following line will do this ...
#dt_ecmwf_loc = [x-x.dst() for x in dt_ecmwf_loc]
# make local time timezone naive to match datetimes in OzFluxQC
dt_ecmwf_ntz = [x.replace(tzinfo=None) for x in dt_ecmwf_loc]
dt_ecmwf_cor = [x - datetime.timedelta(hours=1.5) for x in dt_ecmwf_ntz]

In [9]:
# Downwelling shortwave in ERA-I is a cummulative value that is reset to 0
# at 0300 and 1500 UTC.  Here we convert the cummulative values to
# 3 hourly values.
Fsd_erai_3hr = numpy.ediff1d(Fsd_accum,to_begin=0)
idx = numpy.where((hour_utc==3)|(hour_utc==15))[0]
Fsd_erai_3hr[idx] = Fsd_accum[idx]
Fsd_erai_3hr = Fsd_erai_3hr/(3*3600)

In [ ]:
fig=plt.figure()
plt.plot(dt_ecmwf,Fsd_accum,'b.')
plt.show()

In [10]:
tower_name = "../Sites/Whroo/Data/Portal/Whroo_2014_L3.nc"
tower_file = netCDF4.Dataset(tower_name)
tower_time = tower_file.variables["time"][:]
time_units = getattr(tower_file.variables["time"],"units")
dt_tower =  netCDF4.num2date(tower_time,time_units)
Fsd_tower = tower_file.variables["Fsd"][:,0,0]

In [16]:
alt_solar_3hr = numpy.array([pysolar.GetAltitude(site_latitude,site_longitude,dt) for dt in dt_ecmwf_utc])

In [19]:
fig=plt.figure()
plt.plot(dt_tower,Fsd_tower,'b-')
plt.plot(dt_ecmwf_cor,Fsd_erai_3hr,'r+')
plt.plot(dt_ecmwf_cor,1000*numpy.sin(numpy.deg2rad(alt_solar_3hr)),'g^')
plt.show()

In [ ]:
# Now we interpolate from the 3 hourly ERA-I time step to the tower time step using the solar altitude.
Fsd_erai_1hr = numpy.zeros(len(Fsd_erai_3hr)*3)
idx = numpy.array(range(0,len(Fsd_erai_3hr)))
Fsd_erai_1hr[idx*3] = Fsd_erai_3hr[idx]
Fsd_erai_1hr[idx*3+1] = Fsd_erai_3hr[idx]
Fsd_erai_1hr[idx*3+2] = Fsd_erai_3hr[idx]
# now get the 1 hour datetime series
dt_erai_1hr = [x for x in qcutils.perdelta(dt_ecmwf_cor[0],dt_ecmwf_cor[-1],datetime.timedelta(minutes=60))]
#dt_erai_1hr = [dt_ecmwf_cor[0]-datetime.timedelta(minutes=60)]+dt_erai_1hr
#dt_erai_1hr = dt_erai_1hr+[dt_ecmwf_cor[-1]+datetime.timedelta(minutes=60)]

In [ ]:
fig=plt.figure()
plt.plot(dt_tower,Fsd_tower,'b-')
plt.plot(dt_erai_1hrb,Fsd_erai_1hr,'r+')
plt.show()

In [ ]:
# get the solar altitude, we will use this later to interpolate the ERA Interim
# data from the ERA-I 3 hour time step to the tower time step.
alt_solar_3hr = numpy.array([pysolar.GetAltitude(site_latitude,site_longitude,dt) for dt in dt_ecmwf])
idx = numpy.where(alt_solar_3hr<=0)[0]
alt_solar_3hr[idx] = float(0)
alt_solar_1hr = numpy.zeros(len(alt_solar_3hr)*3)
idx = numpy.array(range(0,len(alt_solar_3hr)))
alt_solar_1hr[idx*3] = alt_solar_3hr[idx]
alt_solar_1hr[idx*3+1] = alt_solar_3hr[idx]
alt_solar_1hr[idx*3+2] = alt_solar_3hr[idx]

In [ ]:
# now get the true solar altitude
dt_erai_utc_1hr = [x for x in qcutils.perdelta(dt_ecmwf[0],dt_ecmwf[-1],datetime.timedelta(minutes=60))]
#dt_erai_utc_1hr = [dt_ecmwf[0]-datetime.timedelta(minutes=60)]+dt_erai_utc_1hr
#dt_erai_utc_1hr = dt_erai_utc_1hr+[dt_ecmwf[-1]+datetime.timedelta(minutes=60)]
alt_solar = numpy.array([pysolar.GetAltitude(site_latitude,site_longitude,dt) for dt in dt_erai_utc_1hr])
idx = numpy.where(alt_solar<=0)[0]
alt_solar[idx] = float(0)

In [ ]:
fig=plt.figure()
plt.plot(dt_ecmwf_cor,alt_solar_3hr)
plt.plot(dt_erai_1hr,alt_solar_1hr,'r+')
plt.plot(dt_erai_1hr,alt_solar,'g^')
plt.show()

In [ ]:
fig=plt.figure()
plt.plot(dt_ecmwf_cor,1000*numpy.sin(numpy.deg2rad(alt_solar_3hr)),'b.')
plt.plot(dt_erai_1hr,1000*numpy.sin(numpy.deg2rad(alt_solar)),'b^')
plt.plot(dt_ecmwf_cor,Fsd_erai_3hr,'r+')
plt.show()

In [ ]:
ratio = numpy.sin(numpy.deg2rad(alt_solar))/numpy.sin(numpy.deg2rad(alt_solar_1hr))
Fsd_erai_1hr = ratio*Fsd_erai_1hr
fig=plt.figure()
plt.plot(dt_tower,Fsd_tower,'b-')
plt.plot(dt_erai_1hr,Fsd_erai_1hr,'r+')
plt.ylim([0,1200])
plt.show()

In [ ]:
from scipy.interpolate import InterpolatedUnivariateSpline
order = 1
